In [1]:
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utilities import *

## 7 Incorporating Conditional Information

### 7.1 Approach I

#### Basic problem:

\begin{equation}
\min_{M_1\geq0}{\mathbb E}\left[M_1g(X_1)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M_1 \log M_1\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M_1 f(X_1)\right] = 0, \\
&{\mathbb E}\left[M_1\right] = 1.
\end{align*}

#### Dual problem:

For computational purposes, we solve the dual problem after minimizing over $M_1$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X_1) + \hat{\lambda} \cdot f(X_1)B^j \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

### 7.2 Approach II

#### Basic problem:

\begin{equation}
\min_{M_1\geq0}{\mathbb E}\left[M_1g(X_1)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M_1 \log M_1\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M_1 {f(X_1)}\right] = 0, \\
&{\mathbb E}\left[B^j M_1-B^j\right] = 0.
\end{align*}

#### Dual problem:

??

### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M_1\geq0}{\mathbb E}\left[M_1g(X_1)\mid\mathfrak{F}_0\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M_1 \log M_1\mid\mathfrak{F}_0\right] \leq \kappa, \\
&{\mathbb E}\left[M_1  f(X_1)\mid\mathfrak{F}_0\right] = 0, \\
&{\mathbb E}\left[M_1\mid\mathfrak{F}_0\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M_1$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X_1) + \hat{\lambda} \cdot f(X_1) \right] \right)\mid\mathfrak{F}_0\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [10]:
# Set ξ
ξ = 1.

# Set model paramters
tol = 1e-8
max_iter = 1000

solver = intertemporal_div_constraint(tol,max_iter)

time_start = time.time() 
result = solver.iterate(ξ)

# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))
print("Numer of iterations: %s ---" % result['count'])

# Print converged parameter results
print("\n")
print("--- Converged parameter vlues ---")
print("ϵ: %s" % result['ϵ'])
print("e: %s" % result['e'])
print("λ: %s" % result['λ'])
print("μ: %s" % result['μ'])

# Print E[M|state k]
print("\n")
print("--- Check ---")
print("E[M|state 1] = %s " % result['E_M_cond'][0])
print("E[M|state 2] = %s " % result['E_M_cond'][0])
print("E[M|state 3] = %s " % result['E_M_cond'][0])

# Print transition probability matrix
print("\n")
print("--- Transition Probability Matrix ---")
print(result['P'])

# Print stationary distribution
print("\n")
print("--- Stationary Distribution ---")
print(result['π'])

# Print conditional relative entropy
print("\n")
print("--- Conditional Relative Entropy ---")
print("E[MlogM|state 1] = %s " % result['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result['RE_cond'][2])

# Print unconditional relative entropy
print("\n")
print("--- Unconditional Relative Entropy ---")
print("E[MlogM] = %s " % result['RE'])

# Print conditional moment bound
print("\n")
print("--- Conditional Moment Bound ---")
print("E[Mg(X)|state 1] = %s " % result['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result['moment_bound_cond'][2])

# Print unconditional moment bound
print("\n")
print("--- Unconditional Moment Bound ---")
print("E[Mg(X)] = %s " % result['moment_bound'])

--- Iteration Ends ---
ξ = 1.0
Time spent: 0.4221 seconds ---
Numer of iterations: 181 ---


--- Converged parameter vlues ---
ϵ: 0.9662646553757464
e: [1.         0.44191823 0.19224322]
λ: [ 1.204626    0.79533031 -0.31077119 -0.76068647  1.4266823   0.57325725
  0.66139294 -5.97034234  2.68478972 -0.68487362 -3.48385636 -7.80332626]
μ: 0.034317511924379644


--- Check ---
E[M|state 1] = 0.9999999995450543 
E[M|state 2] = 0.9999999995450543 
E[M|state 3] = 0.9999999995450543 


--- Transition Probability Matrix ---
[[0.97954794 0.02045206 0.        ]
 [0.08402222 0.88101521 0.03496255]
 [0.         0.1751475  0.82485247]]


--- Stationary Distribution ---
[0.77399257 0.18839952 0.03760792]


--- Conditional Relative Entropy ---
E[MlogM|state 1] = 0.012404187908431235 
E[MlogM|state 2] = 0.06706699447967936 
E[MlogM|state 3] = 0.1681934923932745 


--- Unconditional Relative Entropy ---
E[MlogM] = 0.02856154544655825 


--- Conditional Moment Bound ---
E[Mg(X)|state 1] = 0.005211719565

In [14]:
# Solve the minimization problems over a grid of ξ
tol = 1e-7
max_iter = 1000

solver = intertemporal_div_constraint(tol,max_iter)

# Grid for ξ
ξ_grid = np.arange(.01,1.,.01)
μs = np.zeros_like(ξ_grid)
REs = np.zeros_like(ξ_grid)
bounds = np.zeros_like(ξ_grid)
ϵs = np.zeros_like(ξ_grid)

time_start = time.time() 
for i in range(len(ξ_grid)):
    ξ = ξ_grid[i]
    result = solver.iterate(ξ)
    μs[i] = result['μ']
    REs[i] = result['RE']
    bounds[i] = result['moment_bound']
    ϵs[i] = result['ϵ']

print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

Time spent: 31.5086 seconds ---


In [15]:
# Plots for μ and RE
fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Scatter(x=ξ_grid, y=ϵs, name='ϵ'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=ξ_grid, y=REs, name='RE'),
    row=1, col=2
)
fig.update_layout(height=400, width=1000,title_text='Plots over ξ')
fig.update_xaxes(rangemode="tozero",title_text='ξ')
fig.update_yaxes(rangemode="tozero")
fig.show()